In [9]:
import pandas as pd
import numpy as np

In [10]:
code_folder="../codebook/"
coder2_file=code_folder+'re_code.xlsx' 
coder1_file=code_folder+'round2.xlsx'
code_file=code_folder+"coding.xlsx"

coder1_data=pd.read_excel(coder1_file).replace(np.nan, '', regex=True)
coder2_data=pd.read_excel(coder2_file).replace(np.nan, '', regex=True)
code_data=pd.read_excel(code_file).replace(np.nan, '', regex=True)

# refer to sec 6, codebook

In [12]:
def read_xlsx(fname):
    global keys

    l = []
    reader = pd.read_excel(fname, header=None).replace(np.nan, '', regex=True)
    for _, row in reader.iterrows():
        row = row.values.tolist()          
        if not keys: 
            keys=row            
            continue
        l.append(dict(zip(keys,row)))    
    return l

keys = []

global keys

pri_l = read_xlsx(coder1_file)
sec_l = read_xlsx(coder2_file)

id_k = keys[0]
q_k = keys[1]

pri_d = {d[id_k]:d for d in read_xlsx(coder1_file)[1:]}
sec_d = {d[id_k]:d for d in read_xlsx(coder2_file)[1:]}
num_code_columns=86

overlap = {}
pri_used = {}
sec_used = {}
codes = [f"c{x}" for x in range (1, num_code_columns+1)]
tot_pri = 0
tot_sec = 0
tot = 0
for sk in sec_d: 
 
    s = sec_d[sk] 
 
    p = pri_d[s[id_k]]      
    sec_codes = set(s[c] for c in codes if
                    c in s and s[c] != ''
#                        and not "->" in s[c]
    )
    pri_codes = set(p[c] for c in codes if
                    c in p and p[c] != '' 
#                        and not "->" in p[c]
    )
    if any(sec_codes) and any(pri_codes):
        tot_sec += len(sec_codes)
        tot_pri += len(pri_codes)
        for c in sec_codes.intersection(pri_codes):
            overlap[c] = overlap.get(c,0)+1

        for sc in sec_codes:
            sec_used[sc] = sec_used.get(sc,0)+1

        for pc in pri_codes:
            pri_used[pc] = pri_used.get(pc,0)+1

pri_used = {k:pri_used[k]/tot_pri for k in pri_used}
sec_used = {k:sec_used[k]/tot_sec for k in sec_used}
overlap = {k:overlap[k]/max(tot_pri,tot_sec) for k in overlap}
for c in pri_used:
    if not c in overlap:
        overlap[c]=0.0

for c in sec_used:
    if not c in overlap:
        overlap[c]=0.0
p_a = sum(overlap.values())

p_c = sum(pri_used.get(c,0.0)*sec_used.get(c,0.0) for c in set(pri_used.keys()).union(set(sec_used.keys())))
   
irr = 1 - (1-p_a)/(1-p_c)
# print('irr is',irr)
keys = [] #reset global keys back to empty